In [1]:
import tensorflow as tf 
import numpy as np
import cv2 as cv
import matplotlib.pyplot as plt
import scipy.fft as sc

In [2]:
mnist = tf.keras.datasets.mnist
(x_train, y_train), (x_test, y_test) = mnist.load_data()

In [3]:
def rescaleframe(frame, scale = 0.75):
    width = int(frame.shape[1] * scale)
    height = int(frame.shape[0] * scale)
    dimensions = (width, height)
    return cv.resize(frame, dimensions, interpolation=cv.INTER_AREA)

In [4]:
def centroide(image):
    length = image.shape[0]
    vec = np.linspace(0, length-1, length)
    col_signal = np.sum(image, axis=0)/255 #columna
    row_signal = np.sum(image, axis=1)/255 #fila
    Cx = np.dot(vec, col_signal) / sum(col_signal)
    Cy = np.dot(vec, row_signal) / sum(row_signal)
    return Cx, Cy

In [5]:
def sign(img):
    img = rescaleframe(img, scale=1.5)
    img_blur = cv.blur(img, ksize=(3, 3))
    img_can = cv.Canny(img_blur, 40, 100)
    contours, lista = cv.findContours(img_can, mode=cv.RETR_EXTERNAL, method=cv.CHAIN_APPROX_NONE)
    
    dist = []
    for con in contours:
        dist.append(np.sum(con))
    max_pos = np.argmax(dist)
    main_signal = contours[max_pos].squeeze()
    
    Cx, Cy = centroide(img)
    x, y = np.array_split(main_signal, 2, axis=1)
    mag_x = np.power(Cx-x, 2)
    mag_y = np.power(Cy-y, 2)
    mag = np.power(mag_x + mag_y, 0.5)
    return mag

In [28]:
X = []
Y = []
contador = {}
num_images_per_num = 1000
for i in range(0, 10):
    idx = str(i)
    contador[idx] = 0

for i, x in enumerate(x_train):
    y = y_train[i]
    idx = str(y)
    if contador[idx] < 1000:
        X.append(x)
        Y.append(y)
        contador[idx] = contador[idx] + 1
    else:
        continue

In [29]:
def one_hot_encoder(idx):
    vec = []
    for i in range(0, 10):
        if i == idx:
            vec.append(1)
        else:
            vec.append(0)
    return vec


In [30]:
target = [one_hot_encoder(Y_value) for Y_value in Y]

In [31]:
data = np.array(X)
target = np.array(target)

In [226]:
data = np.reshape(data, (-1, 784))

In [233]:
ann = tf.keras.models.Sequential()
ann.add(tf.keras.layers.Dense(units=128, activation="relu"))
ann.add(tf.keras.layers.Dense(units=64, activation="relu"))
ann.add(tf.keras.layers.Dense(units=32, activation="relu"))
ann.add(tf.keras.layers.Dense(units=10, activation="sigmoid"))
ann.compile(optimizer="adam", loss="binary_crossentropy", metrics="accuracy")

In [234]:
ann.fit(data, target, batch_size=4, epochs=5, verbose=1)

Epoch 1/5
2500/2500 [==============================] - 4s 1ms/step - loss: 0.8536 - accuracy: 0.6590
Epoch 2/5
2500/2500 [==============================] - 3s 1ms/step - loss: 0.0753 - accuracy: 0.8993
Epoch 3/5
2500/2500 [==============================] - 3s 1ms/step - loss: 0.0511 - accuracy: 0.9255
Epoch 4/5
2500/2500 [==============================] - 3s 1ms/step - loss: 0.0400 - accuracy: 0.9456
Epoch 5/5
2500/2500 [==============================] - 3s 1ms/step - loss: 0.0352 - accuracy: 0.9488


In [92]:
data_sign = []
X = np.array(X)
for image in X:
    image_sign = sign(image)

    image_sign_resize = []
    T = len(image_sign)

    for i in range(0, 40):
        step = round(i*T/40)
        image_sign_resize.append(image_sign[step])
    data_sign.append(image_sign_resize)

In [93]:
data_sign = np.array(data_sign)
data_sign.shape

(10000, 40, 1)

In [95]:
data_sign = np.reshape(data_sign ,(-1, 40))

In [100]:
ann = tf.keras.models.Sequential()
ann.add(tf.keras.layers.Dense(units=32, activation="relu"))
ann.add(tf.keras.layers.Dense(units=16, activation="relu"))
ann.add(tf.keras.layers.Dense(units=10, activation="sigmoid"))
ann.compile(optimizer="adam", loss="binary_crossentropy", metrics="accuracy")

In [102]:
ann.fit(data_sign, target, batch_size=4, epochs=5, verbose=1)

Epoch 1/5
5000/5000 [==============================] - 7s 1ms/step - loss: 0.1634 - accuracy: 0.6785
Epoch 2/5
5000/5000 [==============================] - 7s 1ms/step - loss: 0.1574 - accuracy: 0.6920
Epoch 3/5
5000/5000 [==============================] - 7s 1ms/step - loss: 0.1536 - accuracy: 0.6962
Epoch 4/5
5000/5000 [==============================] - 7s 1ms/step - loss: 0.1491 - accuracy: 0.7117
Epoch 5/5
5000/5000 [==============================] - 7s 1ms/step - loss: 0.1463 - accuracy: 0.7131


In [105]:
a = np.zeros((10, 5))

In [112]:
b = np.ones((10, 3))

In [113]:
np.concatenate((a, b), axis=1)

array([[0., 0., 0., 0., 0., 1., 1., 1.],
       [0., 0., 0., 0., 0., 1., 1., 1.],
       [0., 0., 0., 0., 0., 1., 1., 1.],
       [0., 0., 0., 0., 0., 1., 1., 1.],
       [0., 0., 0., 0., 0., 1., 1., 1.],
       [0., 0., 0., 0., 0., 1., 1., 1.],
       [0., 0., 0., 0., 0., 1., 1., 1.],
       [0., 0., 0., 0., 0., 1., 1., 1.],
       [0., 0., 0., 0., 0., 1., 1., 1.],
       [0., 0., 0., 0., 0., 1., 1., 1.]])

In [229]:
data_data_sign = np.concatenate((data, data_sign), axis=1)

In [230]:
ann = tf.keras.models.Sequential()
ann.add(tf.keras.layers.Dense(units=128, activation="relu"))
ann.add(tf.keras.layers.Dense(units=64, activation="relu"))
ann.add(tf.keras.layers.Dense(units=32, activation="relu"))
ann.add(tf.keras.layers.Dense(units=10, activation="sigmoid"))
ann.compile(optimizer="adam", loss="binary_crossentropy", metrics="accuracy")

In [232]:
ann.fit(data_data_sign, target, batch_size=2, epochs=5, verbose=1)

Epoch 1/5
5000/5000 [==============================] - 9s 2ms/step - loss: 0.0502 - accuracy: 0.9329
Epoch 2/5
5000/5000 [==============================] - 9s 2ms/step - loss: 0.0413 - accuracy: 0.9430
Epoch 3/5
5000/5000 [==============================] - 9s 2ms/step - loss: 0.0384 - accuracy: 0.9507
Epoch 4/5
5000/5000 [==============================] - 9s 2ms/step - loss: 0.0328 - accuracy: 0.9557
Epoch 5/5
5000/5000 [==============================] - 9s 2ms/step - loss: 0.0331 - accuracy: 0.9550


In [209]:
Hu = []
for image in X:
    moms = cv.moments(image)
    Hu_moms = cv.HuMoments(moms)
    Hu_moms = Hu_moms.squeeze()
    Hu.append(Hu_moms)
Hu = np.array(Hu)

#norm
min_v = np.min(Hu, axis=0)
max_v = np.max(Hu, axis=0)
Hu = (Hu - min_v)/(max_v-min_v)

In [212]:
data_sign_Hu = np.concatenate((data_sign, Hu), axis=1)

In [217]:
ann = tf.keras.models.Sequential()
ann.add(tf.keras.layers.Dense(units=40, activation="relu"))
ann.add(tf.keras.layers.Dense(units=20, activation="relu"))
ann.add(tf.keras.layers.Dense(units=10, activation="sigmoid"))
ann.compile(optimizer="adam", loss="binary_crossentropy", metrics="accuracy")

In [218]:
ann.fit(data_sign_Hu, target, batch_size=4, epochs=5, verbose=1)

Epoch 1/5
2500/2500 [==============================] - 3s 964us/step - loss: 0.3900 - accuracy: 0.3813
Epoch 2/5
2500/2500 [==============================] - 2s 915us/step - loss: 0.1989 - accuracy: 0.6019
Epoch 3/5
2500/2500 [==============================] - 2s 926us/step - loss: 0.1794 - accuracy: 0.6531
Epoch 4/5
2500/2500 [==============================] - 2s 918us/step - loss: 0.1638 - accuracy: 0.6861
Epoch 5/5
2500/2500 [==============================] - 2s 916us/step - loss: 0.1543 - accuracy: 0.7114


In [211]:
data_sign_Hu.shape

(10000, 47)

In [219]:
data_data_sign_Hu = np.concatenate((data, data_sign_Hu), axis=1)

In [223]:
ann = tf.keras.models.Sequential()
ann.add(tf.keras.layers.Dense(units=128, activation="relu"))
ann.add(tf.keras.layers.Dense(units=64, activation="relu"))
ann.add(tf.keras.layers.Dense(units=32, activation="relu"))
ann.add(tf.keras.layers.Dense(units=10, activation="sigmoid"))
ann.compile(optimizer="adam", loss="binary_crossentropy", metrics="accuracy")

In [224]:
ann.fit(data_sign_Hu, target, batch_size=4, epochs=5, verbose=1)

Epoch 1/5
2500/2500 [==============================] - 3s 1ms/step - loss: 0.2578 - accuracy: 0.4505
Epoch 2/5
2500/2500 [==============================] - 3s 1ms/step - loss: 0.1638 - accuracy: 0.6704
Epoch 3/5
2500/2500 [==============================] - 3s 1ms/step - loss: 0.1384 - accuracy: 0.7225
Epoch 4/5
2500/2500 [==============================] - 3s 1ms/step - loss: 0.1299 - accuracy: 0.7421
Epoch 5/5
2500/2500 [==============================] - 3s 1ms/step - loss: 0.1192 - accuracy: 0.7713


In [225]:
Hu

array([[2.75554023e-01, 4.26457853e-02, 8.61715176e-03, ...,
        1.81695170e-01, 1.58225696e-01, 7.67909485e-01],
       [2.08786893e-01, 1.85816592e-02, 2.07087297e-03, ...,
        1.81687653e-01, 1.58255127e-01, 7.68004300e-01],
       [4.61635369e-01, 3.78236155e-02, 2.70804790e-01, ...,
        1.83432025e-01, 1.63361146e-01, 7.58925326e-01],
       ...,
       [2.13504735e-01, 8.88137437e-03, 4.21736465e-03, ...,
        1.81686085e-01, 1.57953786e-01, 7.67990207e-01],
       [1.51424233e-01, 2.17844714e-02, 6.46996059e-04, ...,
        1.81687595e-01, 1.58201423e-01, 7.67991332e-01],
       [1.87851711e-01, 2.72394671e-02, 4.28734472e-03, ...,
        1.81690997e-01, 1.58215673e-01, 7.67961987e-01]])